In [1]:
import os
import random
import math
import time
import uuid

from tqdm.notebook import tqdm
import numpy as np
from matplotlib.image import imread
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers, losses, initializers

from tensorflow.keras.preprocessing.image import save_img

In [2]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
IMG_SIDE_LENGTH = 224

init_norm = initializers.RandomNormal(mean=0., stddev=1.)

### Building the Discriminator

In [4]:
def build_discriminator(w_init=None):
    D = models.Sequential(name="Discriminator")
    
    # 1st convolutional layer
    D.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu',
                        input_shape=(IMG_SIDE_LENGTH, IMG_SIDE_LENGTH, 3),
                        kernel_initializer=w_init, bias_initializer=w_init))
    
    D.add(layers.Dropout(0.3))
    D.add(layers.MaxPooling2D((2, 2)))
    
    # 2nd convolutional layer
    D.add(layers.Conv2D(64, (3, 3), activation='relu', 
                       kernel_initializer=w_init, bias_initializer=w_init))
    
    D.add(layers.Dropout(0.3))
    D.add(layers.MaxPooling2D((2, 2)))
    
    D.add(layers.Flatten())
    
    D.add(layers.Dense(64, activation='sigmoid', 
                      kernel_initializer=w_init, bias_initializer=w_init))
    
    D.add(layers.Dropout(0.5))
    
    D.add(layers.Dense(1, activation='sigmoid',
                      kernel_initializer=w_init, bias_initializer=w_init))
    
    return D

In [5]:
#D = build_discriminator()
D = build_discriminator(init_norm)
D.summary()

Model: "Discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 110, 110, 32)      2432      
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 53, 53, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 43264)           

### Building the Generator

In [6]:
def build_generator(w_init=None):
    G = models.Sequential(name="Generator")
    
    G.add(layers.Dense(7*7*256, activation='relu', input_shape=(100,),
                      kernel_initializer=w_init, bias_initializer=w_init))
    G.add(layers.BatchNormalization())

    G.add(layers.Reshape((7, 7, 256)))

    G.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu',
                                kernel_initializer=w_init, bias_initializer=w_init))
    G.add(layers.BatchNormalization())

    G.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu',
                                kernel_initializer=w_init, bias_initializer=w_init))
    G.add(layers.BatchNormalization())
    
    G.add(layers.Conv2DTranspose(128, (3, 3), strides=(4, 4), padding='same', activation='relu',
                                kernel_initializer=w_init, bias_initializer=w_init))
    G.add(layers.BatchNormalization())
    
    G.add(layers.Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same', activation='sigmoid',
                                kernel_initializer=w_init, bias_initializer=w_init))
     
    return G

In [7]:
G = build_generator(init_norm)
G.summary()

Model: "Generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 12544)             1266944   
_________________________________________________________________
batch_normalization (BatchNo (None, 12544)             50176     
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 64)        147520    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 128)       51

### Example of a generated image

In [ ]:
noise = np.random.normal(size=(1, 100))
print(noise.shape)

generated_image = np.array(G(noise, training=False))
print(generated_image.shape)

#gen_re = generated_image.reshape((224, 224, 3))
#print(gen_re.mean(), gen_re.max(), gen_re.min())

plt.imshow(generated_image[0, :, :, :])

### Example of a prediction by the Discriminator

In [ ]:
prediction = D(generated_image)
print(prediction)

### Defining parameters for training

In [ ]:
EPOCHS = 50
NOISE_DIM = 100
D_TRAIN_STEPS = 3
BATCH_SIZE = 96
MINIBATCH_SIZE = int(BATCH_SIZE / D_TRAIN_STEPS)

disc_loss_func = losses.BinaryCrossentropy(from_logits=False)
gen_loss_func = losses.BinaryCrossentropy(from_logits=False)

disc_optimizer = optimizers.SGD(learning_rate=0.001)
gen_optimizer = optimizers.SGD(learning_rate=0.001)

### Generator Loss Function

In [ ]:
def generator_loss(predictions):
    return gen_loss_func(tf.ones_like(predictions), predictions)

### Discriminator Loss Function

In [ ]:
def discriminator_loss(real_pred, gen_pred):
    real_loss = disc_loss_func(tf.ones_like(real_pred), real_pred)
    fake_loss = disc_loss_func(tf.zeros_like(gen_pred), gen_pred)
    total_loss = real_loss + fake_loss
    return 1 / total_loss

### Discriminator training for k steps

In [3]:
@tf.function
def train_step_D_k(generator, discriminator, images):
    
    G, D = generator, discriminator

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        D_loss = 0
        
        ## train only D for k steps
        for k in range(D_TRAIN_STEPS):
            # sample noise
            noise = np.random.normal(size=(MINIBATCH_SIZE, NOISE_DIM))
            # turn noise into generated images
            gen_images = G(noise, training=False)
            # sample real images
            real_images = images[ MINIBATCH_SIZE*k : MINIBATCH_SIZE*(k+1)]
            
            # predict
            real_pred = D(real_images, training=True)
            gen_pred = D(gen_images, training=True)
            
            # calculate Discriminator loss
            D_loss += discriminator_loss(real_pred, gen_pred)
        
        D_loss = D_loss / D_TRAIN_STEPS
        
        ## train only G for 1 step
        # sample noise
        noise = np.random.normal(size=(MINIBATCH_SIZE, NOISE_DIM))
        # turn noise into generated images
        gen_images = G(noise, training=True)
        # predict generated images being real
        gen_pred = D(gen_images, training=False)
        # compute generator loss
        G_loss = generator_loss(gen_pred)

    G_gradients = gen_tape.gradient(G_loss, G.trainable_variables)
    D_gradients = disc_tape.gradient(D_loss, D.trainable_variables)

    gen_optimizer.apply_gradients(zip(G_gradients, G.trainable_variables))
    disc_optimizer.apply_gradients(zip(D_gradients, D.trainable_variables))

    return D_loss, G_loss

### Discriminator and Generator train for 1 step

In [5]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(generator, discriminator, images):
    
    G, D = generator, discriminator

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        disc_tape.watch(D.trainable_variables)
        
        ## train only D for 1 step
        # sample noise
        noise = np.random.normal(size=(BATCH_SIZE, NOISE_DIM))
        # turn noise into generated images
        gen_images = G(noise, training=False)
        # predict
        real_pred = D(images, training=True)
        gen_pred = D(gen_images, training=True)
        # calculate Discriminator loss
        D_loss = discriminator_loss(real_pred, gen_pred)
        
        ## train only G for 1 step
        # sample noise
        noise = np.random.normal(size=(BATCH_SIZE, NOISE_DIM))
        # turn noise into generated images
        gen_images = G(noise, training=True)
        # predict generated images being real
        gen_pred = D(gen_images, training=False)
        # compute generator loss
        G_loss = generator_loss(gen_pred)

    G_gradients = gen_tape.gradient(G_loss, G.trainable_variables)
    D_gradients = disc_tape.gradient(D_loss, D.trainable_variables)

    gen_optimizer.apply_gradients(zip(G_gradients, G.trainable_variables))
    disc_optimizer.apply_gradients(zip(D_gradients, D.trainable_variables))

    return D_loss, G_loss

# Training

In [6]:
def train(generator, discriminator, filename, epochs):
    """
    generator - model object
    discriminator - model object
    filename - name of the .npy file with REAL images
    epochs - number of epochs
    """
    
    SKIP_ERROR = "Cannot load file containing pickled data when allow_pickle=False"
    
    for ep in tqdm(range(epochs)):
        total_images_used = 0
        
        D_ep_losses, G_ep_losses = [], []
        
        start = time.time()
        
        pbar = tqdm(total=100)
        
        with open(filename, 'rb') as f:
            while True:
                try:
                    batch_a = np.load(f)
                    batch_b = np.load(f)
                    real_images_batch = np.concatenate((batch_a, batch_b))
                    total_images_used += real_images_batch.shape[0]
                    
                    #D_step_loss, G_step_loss = train_step(images_batch)
                    D_step_loss, G_step_loss = train_step(generator, discriminator, real_images_batch)
                    
                    D_ep_losses.append(D_step_loss)
                    G_ep_losses.append(G_step_loss)
                    
                    pbar.update(round(BATCH_SIZE/15000 * 100, 2))
                except Exception as e:
                    if "allow_pickle=False" not in str(e):
                        print(e)
                    break
        
        pbar.close()
        
        print('Time for epoch {} is {} sec'.format(ep + 1, time.time()-start))
        print(f"Discriminator loss: {np.average(D_ep_losses)}")
        print(f"Generator loss: {np.average(G_ep_losses)}")
        print(f"Trained on {total_images_used} images")
        
        ## save generated image
        noise = np.random.normal(size=(1, NOISE_DIM))
        gen_img_to_save = G(noise, training=False).numpy().reshape(IMG_SIDE_LENGTH, IMG_SIDE_LENGTH, 3)
        save_img(f"../data/generated/{uuid.uuid4()}.jpg", gen_img_to_save)

In [7]:
with tf.device('/CPU:0'):
    
    G = build_generator(init_norm)
    D = build_discriminator(init_norm)
    
    train(G, D, "../data/ports-all.npy", epochs=10)

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory